# 04 Diversity significance

This notebook examines inter-infant differences and changes in trajectories by analysing diversity metrics acquired from the non-collapsed feature table.
As no outcome measures are linked in this assessment, we use diversity metrics calculated from the non-collapsed feature table to avoid losing data.

<img src="./figures/workflow_diversity_significance.jpg" width="75%">

## Setup
Activate the environment `microbEvolve` before running this Jupiter notebook. 

This step loads all required packages and stores the paths to the scripts and data directories in the variables `scripts_dir` and `data_dir`.

In [1]:
import os
import pandas as pd
from qiime2 import Artifact, Metadata, Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

%matplotlib inline

In [2]:
scripts_dir = "src"
data_dir = "../data"

When looking at alpha diversity, we used the Shannon index rather than Pielou’s evenness, because Shannon captures both richness and evenness. We compared Shannon diversity across sampling timepoints to assess changes in alpha diversity over time.

## Statistical Testing of Alpha Diversity kmer-based

In the k-mer–based analysis, no significant differences in Shannon diversity were detected between timepoints, indicating that overall alpha diversity remained stable across the sampled period.

In [4]:
Visualization.load(f"{data_dir}/processed/shannon_kmer_significance.qzv")

<visualization: Visualization uuid: 3c8dae73-5c4c-4909-ab03-6c78a06baf7e>

## Statistical Testing of Alpha Diversity ASV based

We next examined alpha diversity using core metrics calculated from the ASV-based, non-collapsed feature table. Unlike the k-mer–based analysis, this approach revealed a significant difference in alpha diversity between months 2 and 4 ($p = 0.048$).




In [7]:
Visualization.load(f"{data_dir}/processed/shannon_core_significance.qzv")

<visualization: Visualization uuid: 58718f85-0546-464d-a4a7-153d89360aaf>

## PcoA and UMAP

Before performing statistical tests on beta diversity, we first explored the distance matrices using PCoA and UMAP.

The following two scripts perform PCoA and UMAP on the `Bray-Curtis` distance matrices generated during diversity analysis. Dimensionality reduction was carried out using the QIIME plugins `--qiime diversity pcoa` and `--qiime diversity umap`. The scripts also produce .qzv files, which can be viewed with QIIME visualizations if desired.

In [ ]:
! sh {scripts_dir}/pcoa.sh
! sh {scripts_dir}/umap.sh

We explored the samples using several approaches:
- not filtering at all
- selecting only infants with samples at 2, 4, and 6 months
- selecting only the first three samples per infant at each timepoint

We focused on three aspects:
- clustering of samples by infant_id
- clustering of samples by timepoint (2,4 and 6 months)
- clustering of samples by infant_id and timepoint

In this notebook, we include only the key visualizations that we consider most relevant. We focus on data from infants with samples at all three timepoints, which highlights the main takeaways without overcrowding the plots. Additionally, we focus on PCoA plots, as the distances in these visualizations are proportional, while UMAP applies a non-linear transformation, making distances less directly comparable. The full exploration notebooks can be found in `archive/umap_visualization.ipynb` and `archive/pcoa_visualization.ipynb`.

As a sidenote, we did also explore PCoA and UMAP using the Jaccard distances, which did not reveal clear clustering. These visualizations are also available in the exploration notebooks. 
As you will see in the next cells, Bray-Curtis shows clusering, indicating that differences between samples are driven by changes in relative abundances rather than the presence or absence of specific taxa.

### Exporting qza Artifacts

We decided to first export `pcoa_braycurtis.qza` and `umap_braycurtis.qza` into text files, to then load them as dataframes. This allows us to do more flexible plotting and filtering on specific metadata, which may not be possible with the qiime visualization plugin.

In [ ]:
! sh {scripts_dir}/umap_pcoa_exporting.sh

### Load PcoA and Metadata as Dataframe

In [4]:
pcoa_braycurtis = pd.read_csv(
                 f"{data_dir}/raw/umap_pcoa_export/pcoa_braycurtis/ordination.txt",
                 sep='\t',
                 index_col=0,
                 skiprows= 9,
                 header=None
                 )

In [ ]:
metadata = pd.read_csv(
                    f"{data_dir}/raw/metadata.tsv",
                    sep='\t',
                    index_col=0,
                    header=0
                    )

### Filter infants with samples at 2, 4, and 6 months

In [6]:
infants_246 = metadata.groupby("infant_id")["timepoint"].nunique().eq(3)
infants_246_index = infants_246[infants_246].index
metadata_246 = metadata[metadata["infant_id"].isin(infants_246_index)]

### Merge PcoA and Metadata

In [12]:
pcoa_braycurtis_metadata_246_merged = pd.merge(pcoa_braycurtis.iloc[:,0:3], metadata_246, how='inner', left_index=True, right_index=True)
pcoa_braycurtis_metadata_246_merged["infant_id"] = pcoa_braycurtis_metadata_246_merged["infant_id"].astype("category")

### Infant clustering
When coloring by infant_id, distinct clusters are visible. However, multiple clusters also appear within the same infant.

In [8]:
fig = px.scatter_3d(pcoa_braycurtis_metadata_246_merged,
                    x=1,
                    y=2,
                    z=3,
                    color='infant_id',
                    opacity=0.7)

fig.update_layout(margin=dict(l=0, r=0, b=0, t=0))
fig.show()

### Timepoint clustering
When labelling by timepoints, no distinct clusers are forming

In [11]:
fig = px.scatter_3d(pcoa_braycurtis_metadata_246_merged,
                    x=1,
                    y=2,
                    z=3,
                    color='timepoint',
                    opacity=0.7)

fig.update_layout(margin=dict(l=0, r=0, b=0, t=0))
fig.show()

### Infant and Timepoint clusering

When combining `infant_id` and `timepoint` we see that the subclusters that formed within one infant originate from different timepoints.

In [22]:
fig = px.scatter_3d(pcoa_braycurtis_metadata_246_merged,
                    x=1,
                    y=2,
                    z=3,
                    color='infant_id',
                    symbol='timepoint',
                    symbol_map={
                    "2 months": "circle",
                    "4 months": "cross",
                    "6 months": "diamond"},
                    opacity=0.7)

fig.update_layout(margin=dict(l=0, r=0, b=0, t=0))
fig.show()



This is the first indication that the microbiome of the individual infants has changed over time, however we did not find a clear visual trend for the timepoints when excluding the `infant_id` information.

This is the first indication that the microbiome of individual infants changes over time. However, we did not observe a visual trend across timepoints when excluding the `infant_id` information.

# Statistical Testing of Beta Diversity

Due to the longitudinal study design with repeated measures, we need some more sophisticated methods to assess whether groups in our data differ significantly from one another. As we have multiple timepoints per infant and also multiple infants with repeated samples at each timepoint, we cannot assume indepence of either the timepoint or infant groups. Adonis allows us to do *mulitvariate* analysis of variance testing with permutations, whereas other tests like PERMANOVA do not support multivariate designs. 

Sadly, the adonis function exposed in the QIIME plugin q2-longitudinal is currently broken ([Issue #380](https://github.com/qiime2/q2-diversity/issues/380)) and pairwise comparisons of levels in groups is not implemented ([Issue #243](https://github.com/qiime2/q2-diversity/issues/243)) yet.

Luckily, we can use R in Jupyter Notebooks and even pass variables between Python and R with `rpy2`. In the following script, we will investigate whether there are significant differences in microbiome composition between timepoints and infants while controlling for potential confounding or masking effects.

In [2]:
bray_curtis = Artifact.load(f"{data_dir}/raw/boots_kmer_diversity/distance_matrices/braycurtis.qza")
metadata = pd.read_csv(f"{data_dir}/raw/metadata.tsv", sep="\t", index_col=0)

/Users/flurinschindele/miniconda3/envs/microbEvolve/lib/python3.10/site-packages/emperor/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


In [3]:
from skbio import DistanceMatrix

# Convert Artifact to DistanceMatrix object, then to DataFrame
dm = bray_curtis.view(DistanceMatrix)
dm_df = dm.to_data_frame()

# Ensure the order matches
metadata_filtered = metadata.loc[dm_df.index]

In [4]:
%load_ext rpy2.ipython

In [ ]:
%%R
library(vegan)
set.seed(2025)

## Setting up pairwiseAdonis

The next cell just defines the function for the pairwise comparison based on the `pairwiseAdonis` package ([GitHub](https://github.com/pmartinezarbizu/pairwiseAdonis/blob/master/pairwiseAdonis/R/pairwise.adonis2.R)). This was easier than trying to install an R package from Github with conda...

In [6]:
%%R
pairwise.adonis2 <- function(x, data, strata = NULL, nperm=999, ... ) {

#describe parent call function
ststri <- ifelse(is.null(strata),'Null',strata)
fostri <- as.character(x)
#list to store results

#copy model formula
   x1 <- x
# extract left hand side of formula
  lhs <- eval(x1[[2]], environment(x1), globalenv())
  environment(x1) <- environment()
# extract factors on right hand side of formula
  rhs <- x1[[3]]
# create model.frame matrix
  x1[[2]] <- NULL
  rhs.frame <- model.frame(x1, data, drop.unused.levels = TRUE)

# create unique pairwise combination of factors
  co <- combn(unique(as.character(rhs.frame[,1])),2)

# create names vector
  nameres <- c('parent_call')
  for (elem in 1:ncol(co)){
  nameres <- c(nameres,paste(co[1,elem],co[2,elem],sep='_vs_'))
  }
#create results list
  res <- vector(mode="list", length=length(nameres))
  names(res) <- nameres

#add parent call to res
res['parent_call'] <- list(paste(fostri[2],fostri[1],fostri[3],', strata =',ststri, ', permutations',nperm ))


#start iteration trough pairwise combination of factors
 for(elem in 1:ncol(co)){

#reduce model elements
	if(inherits(eval(lhs),'dist')){
	    xred <- as.dist(as.matrix(eval(lhs))[rhs.frame[,1] %in% c(co[1,elem],co[2,elem]),
		rhs.frame[,1] %in% c(co[1,elem],co[2,elem])])
	}else{
	xred <- eval(lhs)[rhs.frame[,1] %in% c(co[1,elem],co[2,elem]),]
	}

	mdat1 <-  data[rhs.frame[,1] %in% c(co[1,elem],co[2,elem]),]

# redefine formula
	if(length(rhs) == 1){
		xnew <- as.formula(paste('xred',as.character(rhs),sep='~'))
		}else{
		xnew <- as.formula(paste('xred' ,
					paste(rhs[-1],collapse= as.character(rhs[1])),
					sep='~'))}

#pass new formula to adonis
	if(is.null(strata)){
	ad <- adonis2(xnew,data=mdat1, ... )
	}else{
	perm <- how(nperm = nperm)
    setBlocks(perm) <- with(mdat1, mdat1[,ststri])
    ad <- adonis2(xnew,data=mdat1,permutations = perm, ... )}

  res[nameres[elem+1]] <- list(ad[1:5])
  }
  #names(res) <- names
  class(res) <- c("pwadstrata", "list")
  return(res)
}


### Method summary
summary.pwadstrata = function(object, ...) {
  cat("Result of pairwise.adonis2:\n")
  cat("\n")
  print(object[1], ...)
  cat("\n")

  cat("Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1\n")
}

## Differences between timepoints

When we want to look into differences between timepoints, we need to consider that the same infants are present in multiple timepoint groups. This makes the timepoint groups no longer independant from each other. Additionally, the differences between timepoints may be masked by the effect of the infants. Using adonis, we can stratify the permutations by `infant_id`, in order to control for it.

As we can see, there seem to be significant differences between the three timepoints indicated by the p-value reported by adonis.

In [7]:
%%R -i dm_df -i metadata_filtered

metadata_filtered$infant_id <- as.factor(metadata_filtered$infant_id)

adonis_strata <- adonis2(as.dist(dm_df) ~ timepoint, data = metadata_filtered, permutations = 999, strata = metadata_filtered$infant_id, by="margin")

print(adonis_strata)

Permutation test for adonis under reduced model
Marginal effects of terms
Blocks:  strata 
Permutation: free
Number of permutations: 999

adonis2(formula = as.dist(dm_df) ~ timepoint, data = metadata_filtered, permutations = 999, by = "margin", strata = metadata_filtered$infant_id)
           Df SumOfSqs      R2      F Pr(>F)    
timepoint   2   2.0281 0.08265 5.2709  0.001 ***
Residual  117  22.5093 0.91735                  
Total     119  24.5374 1.00000                  
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1


We can also look into pairwise comparisons between the three timepoints to identify, which ones are significantly different. Both the comparison of 2 to 6 and 4 to 6 months is significant, whereas the comparison between 2 and 4 months is not significant. Evidently, the microbiome in 6 month old infants seems to be very distinct from the microbiome in younger infants.

In [8]:
%%R -i dm_df -i metadata_filtered

set.seed(2025)

metadata_filtered$infant_id <- as.factor(metadata_filtered$infant_id)

pairwise_adonis_strata <- pairwise.adonis2(as.dist(dm_df) ~ timepoint, data = metadata_filtered, strata = "infant_id", by="margin")

print(pairwise_adonis_strata)

$parent_call
[1] "as.dist(dm_df) ~ timepoint , strata = infant_id , permutations 999"

$`2 months_vs_6 months`
          Df SumOfSqs      R2      F Pr(>F)    
timepoint  1   1.7972 0.10731 9.2562  0.001 ***
Residual  77  14.9507 0.89269                  
Total     78  16.7479 1.00000                  
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

$`2 months_vs_4 months`
          Df SumOfSqs      R2      F Pr(>F)
timepoint  1   0.2013 0.01258 1.0317  0.295
Residual  81  15.8066 0.98742              
Total     82  16.0080 1.00000              

$`6 months_vs_4 months`
          Df SumOfSqs      R2      F Pr(>F)    
timepoint  1   1.0856 0.07074 5.7854  0.001 ***
Residual  76  14.2613 0.92926                  
Total     77  15.3469 1.00000                  
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

attr(,"class")
[1] "pwadstrata" "list"      


## Differences between infants

Similarly, when investigating differences between infants, we must consider that samples were taken at multiple timepoints. The developmental stage (timepoint) likely has a strong effect on the microbiome which could mask individual differences. To control for this, we can stratify the permutations by `timepoint`. This tests whether infants are significantly different from each other while accounting for the shared variance due to age.

In [9]:
%%R -i dm_df -i metadata_filtered

metadata_filtered$infant_id <- as.factor(metadata_filtered$infant_id)
metadata_filtered$timepoint <- as.factor(metadata_filtered$timepoint)


adonis_strata <- adonis2(as.dist(dm_df) ~ infant_id, data = metadata_filtered, permutations = 999, strata = metadata_filtered$timepoint, by="margin")

print(adonis_strata)

Permutation test for adonis under reduced model
Marginal effects of terms
Blocks:  strata 
Permutation: free
Number of permutations: 999

adonis2(formula = as.dist(dm_df) ~ infant_id, data = metadata_filtered, permutations = 999, by = "margin", strata = metadata_filtered$timepoint)
           Df SumOfSqs      R2      F Pr(>F)    
infant_id  16  16.7916 0.68433 13.955  0.001 ***
Residual  103   7.7458 0.31567                  
Total     119  24.5374 1.00000                  
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1


We can also look into pairwise comparisons between infants. This allows us to determine if specific infants have significantly different microbiome compositions compared to others, while still controlling for the timepoint effect. As we have 17 unique infants, there are $136$ pairwise comparisons, so the output is pretty long. I have included it here for completeness sake, but we will not further interpret this. 

In [10]:
%%R -i dm_df -i metadata_filtered

metadata_filtered$infant_id <- as.factor(metadata_filtered$infant_id)

pairwise_adonis_timepoint_strata <- pairwise.adonis2(as.dist(dm_df) ~ infant_id, data = metadata_filtered, strata = "timepoint", by="margin")

print(pairwise_adonis_timepoint_strata)

$parent_call
[1] "as.dist(dm_df) ~ infant_id , strata = timepoint , permutations 999"

$`1_vs_3`
          Df SumOfSqs      R2      F Pr(>F)  
infant_id  1  0.76925 0.40121 6.7004  0.016 *
Residual  10  1.14808 0.59879                
Total     11  1.91733 1.00000                
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

$`1_vs_5`
          Df SumOfSqs      R2      F Pr(>F)   
infant_id  1  0.55611 0.25367 5.4382  0.003 **
Residual  16  1.63614 0.74633                 
Total     17  2.19224 1.00000                 
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

$`1_vs_7`
          Df SumOfSqs      R2      F Pr(>F)  
infant_id  1  0.73304 0.37638 7.2425  0.014 *
Residual  12  1.21458 0.62362                
Total     13  1.94762 1.00000                
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

$`1_vs_10`
          Df SumOfSqs      R2      F Pr(>F)   
infant_id  1  1.24227 0.71862 25.539  0.004 **
Residual  10  0.48

## Effect sizes of both variables

If we include both variables (`timepoint` and `infant_id`), we can even compare the influence of each variable in explaining the variance in the dependent variable - the distance matrix. The $R^2$ metric in permanova/adonis tests describes how much of the variance in the data is explained by our model and the included variables respectively. The $R^2$ values sum up to 1 (not exactly, as we use a marginal test), therefore we can interpret them as ratios of the total variance explained by a variable or the residuals.

As we can see below, `infant_id` explaines much more of the variance in the distance matrix. This is supported by the observations we have made when plotting the distance matrix after dimensionality reduction. There, clustering of samples from the same infant over different timepoints cluster together more closely than samples from the same timepoint of different infants. With these results here, we are able to quantitatively confirm the observations. 

In [11]:
%%R -i dm_df -i metadata_filtered

metadata_filtered$infant_id <- as.factor(metadata_filtered$infant_id)

adonis_result <- adonis2(as.dist(dm_df) ~ timepoint + infant_id, data = metadata_filtered, permutations = 999, by="margin")

print(adonis_result)

Permutation test for adonis under reduced model
Marginal effects of terms
Permutation: free
Number of permutations: 999

adonis2(formula = as.dist(dm_df) ~ timepoint + infant_id, data = metadata_filtered, permutations = 999, by = "margin")
           Df SumOfSqs      R2       F Pr(>F)    
timepoint   2   0.9917 0.04042  7.4152  0.001 ***
infant_id  16  15.7553 0.64209 14.7252  0.001 ***
Residual  101   6.7541 0.27526                   
Total     119  24.5374 1.00000                   
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1
